In [1]:
import pandas as pd #dataframes; para trabalhar com os dataframes
import numpy as np # mathematical operations; operações matemáticas
from os import walk #reading directories and files; para ler diretorios e arquivos
from glob import glob #reading directories and files; para ler diretorios e arquivos

In [2]:
#Connecting to Cloud SDK - to write data into BigQuery tables
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
#(EN-US) I've exported files from Linked Booster into one single folder. Now I will list all files and concat them.

#(PT-BR) Eu organizei os arquivos exportados usando o Linked Booster em uma única pasta. Agora vou listá-los e concatená-los em um só.
list_of_files = glob("/content/drive/MyDrive/Mestrado - UFG/Dissertação/Database/linkedin_data/dados/*")

print("(EN-US) Collected Data Employees from {total_placeholder} B3 companies.\n(PT-BR) Coletados colaboradores trabalhando com Dados em {total_placeholder} empresas da B3"\
      .format(total_placeholder = str(len(list_of_files))))

(EN-US) Collected Data Employees from 152 B3 companies.
(PT-BR) Coletados colaboradores trabalhando com Dados em 152 empresas da B3


In [ ]:
#(EN-US) Performing loop operation into one single dataframe.

#(PT-BR) Performando operação em loop em cada um dos arquivos para adicioná-los em um único dataframe.

# catching the first dataframe as the columns names
df_all = pd.read_csv(list_of_files[0], nrows=0)
for each_file in list_of_files:
  # reading current file in the loop
  df_temporary = pd.read_csv(each_file)

  # concatenating current file in the loop with df_all
  df_all = pd.concat([df_all, df_temporary], axis=0, ignore_index=True)

In [ ]:
df_all.columns

Index(['id', 'name', 'headline', 'location', 'tags', 'notes', 'url', 'email',
       'phoneNumber', 'address', 'twitter1', 'twitter2', 'im1', 'im2', 'im3',
       'website1', 'website2', 'website3'],
      dtype='object')

In [ ]:
#(EN-US) Checking duplicates by id column.

#(PT-BR) Checando duplicadas pela coluna de id.

df_all_no_duplicates = df_all[['id', 'name', 'headline', 'location', 'tags','url']].drop_duplicates().reset_index(drop=True)

In [ ]:
#(EN-US) Defining the list of words that means an Employee has a Data related role.

#(PT-BR) Definindo a lista de palavras que significa que um Funcionário tem uma função relacionada a Dados
is_data_related = ['dados','data','analytics','intel']

In [ ]:
#(EN-US) Mantaining only the data_related Employees.

#(PT-BR) Mantendo apenas os Funcionários data_related.

all_lines = []
for each_line in list(df_all_no_duplicates['headline'].str.strip().str.lower()):
  words_found = []
  for each_word in is_data_related:
    if each_word in str(each_line).lower().strip():
      words_found.append(each_word)
    else:
      continue
  all_lines.append(','.join(words_found))

# Creating a new column with the words found
df_all_no_duplicates['is_data_related'] = all_lines

# Processing the column to show 'IGNORE' when no 'is_data_related' words were found
df_all_no_duplicates['is_data_related'] = np.where(df_all_no_duplicates['is_data_related']=='',
                                                   'IGNORE',
                                                   df_all_no_duplicates['is_data_related'])

# Keeping only the headlines with 'is_data_related' columns
df_all_no_duplicates_only_data_related_headlines = df_all_no_duplicates.query("is_data_related!='IGNORE'").reset_index(drop=True)

In [ ]:
#(EN-US) Normalizing 'tags' column into a new one called 'ticker' in order to relate this dataset with company tax number - CNPJ.

#(PT-BR) Normalizando a coluna 'tags' em uma nova coluna chamada 'ticker' com o objetivo de relacionar esse dataset com o CNPJ das empresas abertas depois.

normalized_tickers = []
for each_tag in list(df_all_no_duplicates_only_data_related_headlines['tags']):
  normalized_tickers.append(str(each_tag.split('-')[0]).strip().upper())

df_all_no_duplicates_only_data_related_headlines['ticker'] = normalized_tickers

In [ ]:
df_all_no_duplicates_only_data_related_headlines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3130 entries, 0 to 3129
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               3130 non-null   object
 1   name             3130 non-null   object
 2   headline         3130 non-null   object
 3   location         3118 non-null   object
 4   tags             3130 non-null   object
 5   url              3130 non-null   object
 6   is_data_related  3130 non-null   object
 7   ticker           3130 non-null   object
dtypes: object(8)
memory usage: 195.8+ KB


In [ ]:
df_all_no_duplicates_only_data_related_headlines.groupby(by=['ticker'], as_index=False)['id'].count().sort_values(by=['id'])

,ticker,id
103,RSUL,1
112,SNSY,1
108,SIMH,1
107,SHUL,1
114,STBP,1
...,...,...
41,CSAN,101
6,AMER,122
96,PSSA,129
14,BBDC,274


In [ ]:
df_all_no_duplicates_only_data_related_headlines.head()

,id,name,headline,location,tags,url,is_data_related,ticker
0,bruno-antunes-3b9aa1199,Bruno Antunes,Engenheiro de Dados | Banco ABC Brasil,"São Paulo, SP",ABCB - dados,https://linkedin.com/in/bruno-antunes-3b9aa1199,dados,ABCB
1,vitor-leite-silva-059603128,Vitor Leite Silva,Cientista de Dados,"São Paulo, SP",ABCB - dados,https://linkedin.com/in/vitor-leite-silva-0596...,dados,ABCB
2,jamille-chagas-852256165,Jamille Chagas,Administração de banco de dados - DBA | SQL Se...,"Santa Rita do Sapucaí, MG",ABCB - dados,https://linkedin.com/in/jamille-chagas-852256165,dados,ABCB
3,prsmedeiros,Paulo Medeiros,MBA BI & Big Data | SFC | Azure | Power BI | S...,"Valinhos, SP",ABCB - dados,https://linkedin.com/in/prsmedeiros,data,ABCB
4,joaovictortalma,João Talma,Engenheiro de dados | Ciência de Dados | Pytho...,"São Paulo, Brazil",ABCB - dados,https://linkedin.com/in/joaovictortalma,dados,ABCB


In [ ]:
df_all_no_duplicates_only_data_related_headlines[df_all_no_duplicates_only_data_related_headlines['name'].str.lower().str.contains('fukumori')]

,id,name,headline,location,tags,url,is_data_related,ticker
934,thiago-fukumori,Thiago Fukumori,Engenheiro de dados Sr.,"São Paulo, SP",COGN - dados,https://linkedin.com/in/thiago-fukumori,dados,COGN


In [ ]:
# Writing into BigQuery
df_all_no_duplicates_only_data_related_headlines.to_gbq(destination_table='linkedin_data.data_related_employees',
                             project_id='mello-dissertation-database',
                             if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 1193.60it/s]


In [5]:
# Reading total employees reported on Linkedin
df_total_employees_by_ticker = pd.read_excel('/content/drive/MyDrive/Mestrado - UFG/Dissertação/Database/linkedin_data/linkedin_data_brazil_employees_qty.xlsx')

In [6]:
df_total_employees_by_ticker.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 272 entries, 0 to 271
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   company_ticker_start    272 non-null    object
 1   linkedin_employees_qty  272 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 4.4+ KB


In [8]:
# Writing into BigQuery
df_total_employees_by_ticker.to_gbq(destination_table='linkedin_data.total_employees',
                             project_id='mello-dissertation-database',
                             if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 825.98it/s]
